# コードなしのデプロイとカスタムコードのデプロイ

本ノートブックでは、
 - Azure Machine Learning ワークスペースを登録する方法を学びます。
 - Azure Container Instances にモデルをデプロイする2通りの方法を学びます。
     - コードなしのデプロイ
     - カスタムコードのデプロイ

※ 参照：[Azure Machine Learning を使用してモデルをデプロイする](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-and-where)

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning の Python SDKをインポートします

<div class="alert alert-info"><h4> Python 環境について</h4><p>
    
Azure Machine Learning Python SDK が導入されている環境であれば、オンプレミスのクライアント PC の Python 環境からもご利用になれます。クライアント PC へのインストール方法については、ドキュメントをご参照ください。
    
**ドキュメント**：[Azure Machine Learning のための開発環境を構成する](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-configure-environment#local)
</p></div>

In [ ]:
import azureml.core

# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

### Azure Machine Learning Workspace との接続

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

### インプットデータとアウトプットデータの指定

scikit-learn の [diabates データセット](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) を用いて学習された `sklearn_regression_model.pkl` を登録してきます。まず、このモデルへのインプットデータとアウトプットデータと指定します。

In [ ]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='sklearn_regression/',
                       overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

## 2. モデル登録とデプロイメント
### モデルの登録

[Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-) を呼び出してモデル登録を行います。

また、モデルのフレームワークやバージョンなどのメタ情報も付与することで **[コードなしのデプロイ](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-and-where#preview-no-code-model-deployment)** の機能を有効にします。

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.19.1',             # Version of scikit-learn used to create the model.
                       sample_input_dataset=input_dataset,
                       sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

### モデルのデプロイ (ノーコード & カスタム)

[Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-) メソッドを用いてモデルをデプロイします。本ノートブックでは、テスト環境でよく使われる Azure Container Instances (ACI) にデプロイします。※ 本番環境では、Azure Kubernetes Service (AKS) の利用を推奨。

### ノーコードデプロイメント (preview)

In [ ]:
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'no-code'

# 既存の推論環境で同じ名前のものを削除する
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

モデルのデプロイが完了したら、Web Service を Call して推論を行います。

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [ 0.03807591,  0.05068012,  0.06169621, 0.02187235, -0.0442235,
         -0.03482076, -0.04340085, -0.00259226, 0.01990842, -0.01764613]
    ],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

### カスタムコードのデプロイメント

全ての機械学習ライブラリに対応する方法です。コードなしのデプロイとは違って推論スクリプトや Python ライブラリなどの情報が必要になります。

Python のライブラリなどの情報は [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) で管理します。

※ 参考:[Azure Machine Learning を使用してトレーニングとデプロイのための環境を再利用します。](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-use-environments)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])

カスタムコードのデプロイにおいては、推論用の Python スクリプトが必須になります。

In [ ]:
with open('score.py') as f:
    print(f.read())

[InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) オブジェクトを[Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-)渡して、カスタムコードのデプロイメントを行います。

In [ ]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.exceptions import WebserviceException

service_name = 'my-custom-env-service'

# 既存の推論環境で同じ名前のものを削除する
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score.py',
                                   source_directory='.',
                                   environment=environment)

service = Model.deploy(ws, service_name, [model], inference_config)
service.wait_for_deployment(show_output=True)

デプロイが完了したら、 [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-) を用いて推論を行います。

In [ ]:
import json

input_payload = json.dumps({
    'data': [
        [ 0.03807591,  0.05068012,  0.06169621, 0.02187235, -0.0442235,
         -0.03482076, -0.04340085, -0.00259226, 0.01990842, -0.01764613]
    ]
})

output = service.run(input_payload)

print(output)

In [ ]:
# Swagger ファイル (json) の取得
service.swagger_uri

### モデルのプロファイリング (Model profiling)

必要な CPU と Memory のスペックをプロファイルすることもできます。

```python
profile = Model.profile(ws, "profilename", [model], inference_config, test_sample)
profile.wait_for_profiling(True)
profiling_results = profile.get_results()
print(profiling_results)
```

### モデルのパッケージ化 (Model packaging)

推論環境 を Docker イメージにパッケージ化することも可能です。Docker イメージは ACR に格納されます。

`Model.package()` に Environment オブジェクトを渡す必要があります。

```python
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)  # Or show_output=False to hide the Docker build logs.
package.pull()
```

Docker イメージをローカルでビルドするために必要な Dockerfile, モデル, およびその他のアセットをダウンロードすることもできます。

```python
package = Model.package(ws, [model], inference_config, generate_dockerfile=True)
package.wait_for_creation(show_output=True)
package.save("./local_context_dir")
```